## Table of Contents
###  [1.Pre-Processing](pre-processing.ipynb)
###  [2.Data Assimilation](Data Assimilation.ipynb)
<div class="toc" style="margin-top: 1em;">
   <ul class="toc-item" id="toc-level0">
      <li><span><a href='#part 2.1' data-toc-modified-id="part 1"><span class="toc-item-num">&nbsp;&nbsp;</span>2.1 Data input</a></span></li>
      <li><span><a href='#part 2.2' data-toc-modified-id="part 2"><span class="toc-item-num">&nbsp;&nbsp;</span>2.2 PFLOTRAN configuration</a></span></li>
      <li><span><a href='#part 2.3' data-toc-modified-id="part 3"><span class="toc-item-num">&nbsp;&nbsp;</span>2.3 EnKF implementation</a></span></li>
</div>
###  [3.Post-Processing](post-processing.ipynb)






<a id='part 2.1'></a>
# 2.1 Data input 
The input data include:
1. geometry configuration and initial conditions for 1-D hydro-thermal momdel that is used to assimilate observed temperatures below river bed
2. settings for EnKF 
3. configuration for parallel implementation on high performance computer


In [ ]:
# library import
library(rhdf5)
library(RCurl) #for merage list
library(xts)
source("./codes/vhg_da_function.R")


nreaz = 100 # number of realizations 
ncore = 48 # number of cores 

time.window = 3600*24 # time window that impacts the estimation of hydraulic conductivty in next time step

init.datum = 5
input.dir = "dainput/" 
obs.coord = c(-0.25)
obs.sd.ratio = 0.01
nobs = length(obs.coord)
cond.mean = 0
cond.sd = 0.5
set.seed(999)
init.cond = 10^rnorm(nreaz,cond.mean,cond.sd)

perm.threshold = c(1e-15,1e-7)

nz = 75
dz = rep(1.5/nz,nz)
z = -1.5+cumsum(dz)-0.5*dz

ntime = 2000
simu.time = seq(3600,3600*ntime,3600)

obs = read.table("dainput/obs.dat")
obs = as.matrix(obs)
obs.time = obs[,1]
obs.interval = 300

##initial
set.seed(0)
init.perm = init.cond*0.001/1000/9.81/24/3600
init.perm.sd = sd(log(init.perm))




<a id='part 2.2'></a>
# 2.2 PFLOTRAN configuration
1. creat individual directory to store PFLOTRAN input file and output file for each realization
2. The function cenkf.mc.input.strata() is aimed to generate input file for each PFLOTRAN simulation by incorporating the updated realizations. 
3. The function cenkf.mc.output.window() is used to generate the simulated ensemble during the 24-hour time window.

In [ ]:
# make directories for each realization for PFLOTRAN files
system("rm -rf pflotran_mc",wait=TRUE,ignore.stdout=TRUE,ignore.stderr=TRUE)        
system("mkdir pflotran_mc",wait=TRUE,ignore.stdout=TRUE,ignore.stderr=TRUE)    
for (ireaz in 1:nreaz)
{
    system(paste("mkdir pflotran_mc/",ireaz,sep=""),
           wait=TRUE,ignore.stdout=TRUE,ignore.stderr=TRUE)            
}

# prepare PFLOTRAN input file for each realization
cenkf.mc.input.strata <- function(forecast.start,forecast.end,collect.times)
{

    pflotranin = readLines("dainput/1dthermal.in")
    ##check checkpoint
    lindex = grep("CHECKPOINT",pflotranin)
    pflotranin[lindex+1] = paste("    TIMES sec ",simu.time[forecast.end])
    ##determine when to restart simulation
    lindex = grep("pflotran-restart.chk",pflotranin)                    
    if(forecast.start==1)
    {
        pflotranin[lindex] = "# RESTART pflotran-restart.chk"
    }else{
        pflotranin[lindex] = paste(" RESTART 1dthermal-",
                                   sprintf("%.4f",simu.time[forecast.start-1]),"sec.chk",sep="")        
    }

    lindex = grep("FINAL_TIME",pflotranin)        
    pflotranin[lindex] = paste("  FINAL_TIME",simu.time[forecast.end],"sec")

    lindex = grep("SNAPSHOT_FILE",pflotranin)        
#    pflotranin[lindex+1] = paste("   TIMES sec", paste(collect.times,collapse=" "))
    pflotranin[lindex+1] = paste("PERIODIC TIME",obs.interval,"sec")
    

    perm.card.length=14
    perm.card.lindex = grep("MATERIAL_PROPERTY Alluvium",pflotranin)-1        
    perm.card = pflotranin[1:perm.card.length+perm.card.lindex]
    perm.value.cardindex = grep("PERM_ISO",perm.card)

    strata.card.lindex = grep("STRATA",pflotranin)-1        
    strata.card = c("STRATA","REGION all")
    strata.card = c(strata.card,paste("  MATERIAL Perm",1,sep=""))
    strata.card = c(strata.card,paste("  START_TIME",0,"sec"))
    strata.card = c(strata.card,paste("  FINAL_TIME",simu.time[1],"sec"))    
    strata.card = c(strata.card,"END")
    strata.card = c(strata.card,"")        
    strata.card.length=length(strata.card)    

    for (ireaz in 1:nreaz)
    {
        strata.card.new = c()
        perm.card.new = c()    
        
        temp.pflotranin = pflotranin
        for (jtime in 1:forecast.end)
            {
                if (jtime==1)
                {
                    strata.card[4] = paste("  START_TIME",0,"sec")
                }else
                {
                    strata.card[4] = paste("  START_TIME",simu.time[jtime-1],"sec")
                }
                strata.card[5] = paste("  FINAL_TIME",simu.time[jtime],"sec")                
                strata.card[3] = paste("  MATERIAL Perm",jtime,sep="")                
                strata.card.new = c(strata.card.new,strata.card)

                perm.card[1] = paste("MATERIAL_PROPERTY Perm",jtime,sep="")
                perm.card[2] = paste("  ID",jtime)                
                perm.card[perm.value.cardindex] = paste("    PERM_ISO",perm.ls[[jtime]][ireaz])
                perm.card.new = c(perm.card.new,perm.card)
            }
        temp.pflotranin = c(temp.pflotranin[1:strata.card.lindex],
                            strata.card.new,
                            temp.pflotranin[-c(1:(strata.card.lindex+strata.card.length))])
                

        temp.pflotranin = c(temp.pflotranin[1:perm.card.lindex],
                      perm.card.new,
                      temp.pflotranin[-c(1:(perm.card.lindex+perm.card.length))])                
        writeLines(temp.pflotranin,paste("pflotran_mc/",ireaz,"/1dthermal.in",sep=""))
    }
    
}

cenkf.mc.output.window <- function(collect.times)
{
    obs.cell = rep(NA,nobs)
    for (iobs in 1:nobs)
    {
        obs.cell[iobs] = which.min(abs(z-obs.coord[iobs]))
    }
    mc_dir="pflotran_mc/"
    
    simu.ensemble = array(NA,c(nreaz,nobs*length(collect.times)))

    for (ireaz in 1:nreaz)
    {
        obs.temp = c()
        for (collect.itime in collect.times)
        {
            obs.temp = c(obs.temp,h5read(paste(mc_dir,ireaz,"/1dthermal.h5",sep=''),
                                         paste("Time:",sprintf("%12.5E",collect.itime),
                                               "s/Temperature [C]"))[obs.cell])
        }
        simu.ensemble[ireaz,] = obs.temp
        H5close()        
    }

    return(simu.ensemble)
}

<a id='part 2.3'></a>
# 2.3 EnKF implementation
1. Implementation of EnKF to assimulate the observed temperatures.
2. Only data in the past 24-hour is kept in the ensemble. 

In [ ]:
perm.ls = list()
perm.ls[[1]] = init.perm
for (itime in 1:ntime)        
{
    tloopbeg = Sys.time()
    print(paste(itime,
                mean(log10(perm.ls[[itime]]*3600*24*9.81*1000*1000)),
                sd(log10(perm.ls[[itime]]*3600*24*9.81*1000*1000))))      


    window.start = which.min(abs(simu.time-(simu.time[itime]-time.window)))

    collect.index = which(
        obs.time>max(simu.time[itime-1],0) &
        obs.time<=simu.time[itime])
    
    collect.times = obs.time[collect.index]
    
    ncollect = length(collect.times)
   
    cenkf.mc.input.strata(max(window.start-1,1),itime,collect.times)
    # system(paste("sh codes/mc.cori.sh",nreaz,ncore),
    #        wait=TRUE,ignore.stdout=TRUE,ignore.stderr=TRUE)
    tshbeg = Sys.time()
    system(paste("sh codes/mc.cori.sh",nreaz,ncore),
           wait=TRUE,ignore.stdout=TRUE,ignore.stderr=TRUE)
    tshend = Sys.time()
    print(paste("shell script costs",as.numeric(difftime(tshend,tshbeg))))

    simu.ensemble = cenkf.mc.output.window(collect.times)

    set.seed(itime)
    obs.sd = obs.sd.ratio*as.numeric(c(t(obs[collect.index,-1])))
    obs.ensemble = array(replicate(nreaz,as.numeric(c(t(obs[collect.index,-1])))),c(nreaz,ncollect*nobs))+
        array(rnorm(nreaz*nobs*ncollect,0,1),
              c(nreaz,nobs*ncollect))*obs.sd

    #update
    state.vector = c()
    for (jtime in window.start:itime)
    {
        state.vector = cbind(state.vector,array(log(perm.ls[[jtime]]),c(nreaz,1)))
    }
    
    cov.state_simu = cov(state.vector,simu.ensemble)
    cov.simu = cov(simu.ensemble,simu.ensemble)
    inv.cov.simuADDobs = solve(cov.simu+obs.sd**2)
    kalman.gain = cov.state_simu %*% inv.cov.simuADDobs
    state.vector = state.vector+(obs.ensemble-simu.ensemble) %*% t(kalman.gain)


    for (jtime in window.start:itime)
    {
        perm.ls[[jtime]] = exp(state.vector[,jtime-window.start+1])
        perm.ls[[jtime]][perm.ls[[jtime]]>perm.threshold[2]] = perm.threshold[2]
        perm.ls[[jtime]][perm.ls[[jtime]]<perm.threshold[1]] = perm.threshold[1]
    }

    ##prepare input for next segment    
    perm.ls[[itime+1]] = perm.ls[[itime]]

    #disturb
    perm = log(perm.ls[[itime+1]])
    set.seed(itime+10^7)    
    perm = perm+rnorm(nreaz,0,max(init.perm.sd^2-sd(perm)^2,0)^0.5)
    perm.ls[[itime+1]] = exp(perm)
    perm.ls[[itime+1]][perm.ls[[itime+1]]>perm.threshold[2]] = perm.threshold[2]
    perm.ls[[itime+1]][perm.ls[[itime+1]]<perm.threshold[1]] = perm.threshold[1]

    
    save(list=c("perm.ls"),file=paste("results/enkf_perm_temp_",itime,".r",sep=""))    
    tloopend = Sys.time()
    print(paste("loop costs",as.numeric(difftime(tloopend,tloopbeg))))
}

save(list=ls(),file="results/enkf.r")
save(list=c("perm.ls"),file="results/enkf_perm_temp.r")